## F.3 Connecting the dots

Now that you have implemented the QFT for one and two qubits, we will focus on implementing a generalized QFT on  qubits. To get an intuition behind the gates involved, it is easier to look at the action of the QFT on the computational basis states, rather than the matrix formulation. The action of the QFT on a -qubit state  is given by

Codercise F.3.1. Implement the QFT for three qubits.

Hint.
Calculate the controlled- gates as follows:

These are familiar gates! Remember too that you can use the qml.ctrl operation to make a controlled gate.

In [2]:
import pennylane as qml

In [3]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    ##################
    # YOUR CODE HERE #
    ################## 

    qml.Hadamard(wires=0)

    qml.ctrl(qml.PhaseShift, (1,2), control_values=(1,0))(np.pi/2, wires=0)
    
    qml.ctrl(qml.PhaseShift, (1,2), control_values=(0,1))(np.pi/4, wires=0)
    
    qml.Hadamard(wires=1)

    qml.ctrl(qml.PhaseShift, (0,2), control_values=(0,1))(np.pi/2, wires=1)
    
    qml.Hadamard(wires=2)

    qml.SWAP(wires=[0,2])

    return qml.state()

Great, now let's go back to the general case for the QFT. We perform the Hadamard operation followed by a sequence of rotations for each qubit. Given a state , the action on one qubit  depends on all of the following qubits  (the less significant qubits). With this, we get our output in reverse order i.e.,

We could correct this easily by reversing the order of qubits by using SWAP gates.

Let's work backwards and first implement an -qubit SWAP program. After that, we will look into implementing rotations to tie it all together to produce the -qubit QFT.

Codercise F.3.2. Implement a circuit that reverses the order of  qubits using SWAP gates. You need only  gates for this.

In [4]:
dev = qml.device('default.qubit', wires=4)

            
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    n = n_qubits - 1
    for i in range(n_qubits//2):
        qml.SWAP(wires=[i, n])
        n -= 1

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    # qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

Now, we will implement the rotations on qubits. Let us take a look at the circuit again and observe the rotations on each qubit.

circuit

We will first take an iterative approach (and in the next exercise, a recursive approach) to implement the rotations. For each qubit, we need to perform a Hadamard operation followed by a loop going over its lesser significant qubits implementing controlled rotations.

You may have used the  gate and the  gate for the QFT on three qubits. These are special cases of a PennyLane gate called the qml.PhaseShift operation; you can make use of the general operation here. Furthermore, since we are using controlled phase gates in the QFT circuit, you can use the qml.ControlledPhaseShift operation directly.

Codercise F.3.3. Implement the circuit that performs the Hadamards and controlled rotations on  qubits using qml.ControlledPhaseShift. The swap_bits operation defined in the exercise above is available to use. The result is the -qubit QFT!

In [5]:
dev = qml.device('default.qubit', wires=4)

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        rotation = np.pi/2
        control = 1
        for j in range(i, n_qubits-1):
            qml.ControlledPhaseShift(rotation, wires=[i+control, i])
            rotation *= .5
            control += 1    
    
@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

You have successfully implemented the QFT! We will now try to implement a recursive solution to the rotations part of QFT as well. Let's take a look at the circuit again.

circuit

For the first qubit, we perform a Hadamard operation followed by a set of rotations with a contribution from the second to the last qubit. For the second set of qubits, we perform a Hadamard operation followed by a reduced set of the same rotations where the contributions come from the third to the last qubits. This goes on until we reach the last qubit where only a Hadamard gate needs to be applied without rotations. Hopefully, this is enough to get you started with a recursive solution!

Codercise F.3.4. Implement the circuit that performs the Hadamard operations and controlled rotations on  qubits recursively. The swap_bits operation defined earlier is available for you.

Hint.
To start implementing recursive solution, it is useful to think of the base case and the recursive case. The recursive case for a QFT on  qubits is a Hadamard gate and a certain number of Controlled- rotations followed by a QFT for  qubits. This goes on until we reach the base case which is a single qubit QFT.

In [6]:
dev = qml.device('default.qubit', wires=4)

def qft_recursive_rotations(n_qubits, wire=0):
    """A circuit that performs the QFT rotations on the specified qubits
        recursively.
        
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
        wire (int): An integer identifying the wire 
                    (or the qubit) to apply rotations on.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    if n_qubits == 1:
        qml.Hadamard(wires=wire)
    else:
        qml.Hadamard(wires=wire)
        rotation = np.pi/2
        control = 1
        for j in range(wire, n_qubits-1):
            qml.ControlledPhaseShift(rotation, wires=[wire+control, wire])
            rotation *= .5
            control += 1 
        qft_recursive_rotations(n_qubits-1, wire+1)    
                
@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_recursive_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

Now that you've learned the inner workings of the QFT, we can use the PennyLane template to perform it. Additionally, if you check out the template's code, you'll see that you essentially implemented it in this module!

Codercise F.3.5. Implement the QFT using qml.QFT.

In [7]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def pennylane_qft(basis_id, n_qubits):
    """A that circuit performs the QFT using PennyLane's QFT template.
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
        n_qubits (int): An integer identifying the 
            number of qubits.
            
    Returns:
        array[complex]: The state after applying the QFT to the qubits.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))

    ##################
    # YOUR CODE HERE #
    ################## 
    qml.QFT(wires=range(n_qubits))
    return qml.state()